# Prepare working dataset for search and vis
- Data from clinicaltrials.gov

In [4]:
import json, os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nltk

## Load json file with parsed information

In [5]:
# paths
path_to_json_file = os.path.abspath('../data/json/')

### json file
# json_file = '/all_parsed_data_json' #name json file  
json_file = '/sample_json' #sample json file for testing  

file = '{}{}.json'.format(path_to_json_file, json_file)

In [6]:
# load json file
df = pd.read_json(file)

In [7]:
df.head()

,nct_id,study_first_submitted,source,brief_title,overall_status,verification_date,study_type,study_first_posted,last_update_submitted,last_update_posted,...,detailed_description/textblock,brief_summary/textblock,location/facility/address/city,location/facility/address/country,location/facility/address/zip,sponsors/lead_sponsor/agency,sponsors/lead_sponsor/agency_class,study_design_info/allocation,study_design_info/intervention_model,study_design_info/primary_purpose
0,NCT00058058,"April 7, 2003",American College of Radiology Imaging Network,Magnetic Resonance Imaging in Women Recently D...,Completed,June 2018,Interventional,"April 9, 2003","June 29, 2018","July 27, 2018",...,\n OBJECTIVES:\n\n - Determine th...,\n RATIONALE: Diagnostic procedures such ...,Little Rock,United States,72205,American College of Radiology Imaging Network,Other,None,Single Group Assignment,Diagnostic
1,NCT01980602,"September 24, 2013",University of Hull,Effect of Exercise on Patients With Claudicati...,Unknown status,November 2013,Interventional,"November 11, 2013","November 4, 2013","November 11, 2013",...,None,\n Title: How does exercise improve the c...,Hull,United Kingdom,HU3 2JZ,University of Hull,Other,Randomized,Single Group Assignment,Treatment
10,NCT01277458,"January 14, 2011",Public Health England,Ethnicity Data in HIV Positive Men Who Have Se...,Unknown status,December 2010,Observational,"January 17, 2011","January 14, 2011","January 17, 2011",...,\n HIV Doctors will be identified at each...,\n Are there differences between the way ...,London,United Kingdom,SE1 7EH,Public Health England,Other,None,None,None
100,NCT01164891,"July 16, 2010",Hoffmann-La Roche,A Pharmacokinetic and Metabolism Study of 14C-...,Completed,September 2017,Interventional,"July 19, 2010","January 11, 2016","February 5, 2016",...,None,"\n This open-label, non-randomized study ...",Zürich,Switzerland,8091,Hoffmann-La Roche,Industry,None,Single Group Assignment,Treatment
1000,NCT03795727,"January 3, 2019",Cairo University,Clinical Evaluation of Sectional Matrix Versus...,Not yet recruiting,February 2019,Interventional,"January 8, 2019","March 10, 2019","March 12, 2019",...,None,\n The aim of this study is to evaluate t...,Cairo,Egypt,11553,Cairo University,Other,Randomized,Parallel Assignment,Treatment


## Clean dataset

In [8]:
# change column type
def data_types(dataframe, cols = [], to_type = ''):
    for col in cols:
        dataframe[col] = df[col].astype(to_type)

In [9]:
columns_dates = ['study_first_submitted', 'last_update_submitted', 'verification_date']
data_types(df, columns_dates, 'datetime64')

In [10]:
# Extract year 
df['year_submitted'] = df['study_first_submitted'].dt.year
df['year_last_updated'] = df['last_update_submitted'].dt.year
df['year_verification'] = df['verification_date'].dt.year

In [11]:
# Remove unnecesary columns
df.drop(columns_dates, axis = 1, inplace=True)

In [12]:
# Remove /n in all df
df = df.replace(r'\n',' ', regex=True)

In [13]:
# Create new column for all text
df['all_text'] = df['source'] + ' ' + df['brief_title'] + ' ' + df['condition'] + ' ' + df['condition_browse/mesh_term'] + ' '+ df['intervention_browse/mesh_term'] + ' '+ df['detailed_description/textblock'] + ' ' + df['brief_summary/textblock']

# All_text in lowercase
df['all_text'] = df['all_text'].str.lower()

# remove extra whitespace
df.all_text = df.all_text.replace('\s+', ' ', regex=True)


In [14]:
# Remove extra white space in summary
df['brief_summary/textblock'] = df['brief_summary/textblock'].replace('\s+', ' ', regex=True)
df['brief_summary/textblock'][0]

' RATIONALE: Diagnostic procedures such as magnetic resonance imaging (MRI) may improve the ability to detect cancer in the unaffected breast of women recently diagnosed with unilateral breast cancer. PURPOSE: Diagnostic trial to determine the effectiveness of MRI in evaluating the unaffected breast of women recently diagnosed with unilateral breast cancer. '

## Preprocess dataset
- Tokenize, Lemmatize / Stem
- Remove stopwords

In [15]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords

porter_stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer() 

In [16]:
# Stemming
# def stem_sentences(text):
#     tokens = text.split()
#     stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
#     return ' '.join(stemmed_tokens)

# df['stems'] = df['all_text'].apply(stem_sentences)

In [17]:
# Lemmatizing
def lemm_sentences(text):
    tokens = text.split()
    lemm_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemm_tokens)

df['lemmas'] = df['all_text'].apply(lemm_sentences)

In [18]:
# Remove stop words [not needed with countvectorizer]
# stop_words = stopwords.words('english')
# df['tokens'] = df['lemmas'].apply(lambda x: [item for item in x.split() if item not in stop_words])

In [19]:
df.lemmas[0:5]

0       american college of radiology imaging network ...
1       university of hull effect of exercise on patie...
10      public health england ethnicity data in hiv po...
100     hoffmann-la roche a pharmacokinetic and metabo...
1000    cairo university clinical evaluation of sectio...
Name: lemmas, dtype: object

## Bags of words

In [20]:
# Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
import re

In [21]:
pat_numbers = lambda x: re.sub(r'(\d)+', '', x.lower())

cv = CountVectorizer(stop_words='english',
                     preprocessor = pat_numbers,
                     max_features = 3000,
                     lowercase = True,
                     max_df = 0.5,
                     ngram_range = (1, 3))

In [22]:
docs = list(df['lemmas'])

X = cv.fit_transform(docs)

In [23]:
len(cv.vocabulary_)

3000

In [24]:
pd.DataFrame(X.toarray(), columns=cv.get_feature_names()).head()

,abdominal,ability,ablation,able,abnormal,abnormality,absence,absence disease,absence disease progression,absorption,...,year,year age,year follow,year old,years,yield,york,young,youth,zinc
0,0,1,0,0,0,0,0,0,0,0,...,2,0,0,2,0,2,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## TF-IDF

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer

In [26]:
transformer = TfidfTransformer()
tweights = transformer.fit_transform(X)
tweights

<7941x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 650886 stored elements in Compressed Sparse Row format>

In [27]:
# turn weights data into a dataframe
tf = pd.DataFrame(tweights.toarray(), columns=cv.get_feature_names())

In [28]:
# Top terms by average tf-idf weight
weights = np.asarray(tweights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cv.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(10)

,term,weight
2814,treatment,0.031892
351,cancer,0.023008
771,disease,0.021017
2408,safety,0.019931
2665,subject,0.019689
801,dose,0.018561
1125,group,0.018109
847,effect,0.017283
661,day,0.017258
2760,therapy,0.016957


In [29]:
# Create new dataframe with nct-id and merge wth tf by index
pd.set_option('display.max_columns', None) 

df_tf = df['nct_id']
df_tf.head()

0       NCT00058058
1       NCT01980602
10      NCT01277458
100     NCT01164891
1000    NCT03795727
Name: nct_id, dtype: object

In [30]:
df_tf = pd.merge(df_tf, tf, left_index=True, right_index=True)

In [31]:
df_tf.head()

nct_id  abdominal   ability  ablation  able  abnormal  abnormality  \
0     NCT00058058        0.0  0.038834       0.0   0.0       0.0          0.0   
1     NCT01980602        0.0  0.000000       0.0   0.0       0.0          0.0   
10    NCT01277458        0.0  0.000000       0.0   0.0       0.0          0.0   
100   NCT01164891        0.0  0.000000       0.0   0.0       0.0          0.0   
1000  NCT03795727        0.0  0.000000       0.0   0.0       0.0          0.0   

      absence  absence disease  absence disease progression  absorption  \
0         0.0              0.0                          0.0         0.0   
1         0.0              0.0                          0.0         0.0   
10        0.0              0.0                          0.0         0.0   
100       0.0              0.0                          0.0         0.0   
1000      0.0              0.0                          0.0         0.0   

      abuse   ac  acceptability  acceptable  accepted    access  accordance  \
0       0.0  0.0            0.0    0.000000       0.0  0.000000         0.0   
1       0.0  0.0            0.0    0.000000       0.0  0.000000         0.0   
10      0.0  0.0            0.0    0.086624       0.0  0.145135         0.0   
100     0.0  0.0            0.0    0.000000       0.0  0.000000         0.0   
1000    0.0  0.0            0.0    0.000000       0.0  0.000000         0.0   

      according  account   accrual   accrued  accrued study  accuracy  \
0           0.0      0.0  0.049962  0.051339       0.053121       0.0   
1           0.0      0.0  0.000000  0.000000       0.000000       0.0   
10          0.0      0.0  0.000000  0.000000       0.000000       0.0   
100         0.0      0.0  0.000000  0.000000       0.000000       0.0   
1000        0.0      0.0  0.000000  0.000000       0.000000       0.0   

      accurate  acetaminophen  achieve  achieved  achieving  acid  acne  \
0          0.0            0.0      0.0       0.0        0.0   0.0   0.0   
1          0.0            0.0      0.0       0.0        0.0   0.0   0.0   
10         0.0            0.0      0.0       0.0        0.0   0.0   0.0   
100        0.0            0.0      0.0       0.0        0.0   0.0   0.0   
1000       0.0            0.0      0.0       0.0        0.0   0.0   0.0   

      acquired  act  acting  action  activated  activation  active  \
0          0.0  0.0     0.0     0.0        0.0         0.0     0.0   
1          0.0  0.0     0.0     0.0        0.0         0.0     0.0   
10         0.0  0.0     0.0     0.0        0.0         0.0     0.0   
100        0.0  0.0     0.0     0.0        0.0         0.0     0.0   
1000       0.0  0.0     0.0     0.0        0.0         0.0     0.0   

      activities  activity    actual  acuity  acupuncture  acute  \
0            0.0  0.000000  0.000000     0.0          0.0    0.0   
1            0.0  0.000000  0.081801     0.0          0.0    0.0   
10           0.0  0.000000  0.000000     0.0          0.0    0.0   
100          0.0  0.000000  0.000000     0.0          0.0    0.0   
1000         0.0  0.028092  0.000000     0.0          0.0    0.0   

      acute myeloid  acute myeloid leukemia   ad  adapted  adaptive  add  \
0               0.0                     0.0  0.0      0.0       0.0  0.0   
1               0.0                     0.0  0.0      0.0       0.0  0.0   
10              0.0                     0.0  0.0      0.0       0.0  0.0   
100             0.0                     0.0  0.0      0.0       0.0  0.0   
1000            0.0                     0.0  0.0      0.0       0.0  0.0   

      added  addiction  adding  addition  additional  additionally  address  \
0       0.0        0.0     0.0       0.0    0.035647           0.0      0.0   
1       0.0        0.0     0.0       0.0    0.000000           0.0      0.0   
10      0.0        0.0     0.0       0.0    0.000000           0.0      0.0   
100     0.0        0.0     0.0       0.0    0.000000           0.0      0.0   
1000    0.0        0.0    

## Test merged dataframes with tf-idf results

In [48]:
import random

def compare_results():
    row = random.randint(0,3000)
    df_tf.drop('nct_id', axis = 1)
    print('Random row: {}'.format(row))
    return df_tf.loc[row], tf.loc[row]

In [52]:
# compare_results()

## Calculate correlation between docs

In [53]:
# calculate similary
similarity = tweights * tweights.T

In [58]:
# Save all data in a dataframe
df_similarity = pd.DataFrame(similarity.toarray())

In [62]:
# Find more similar documents of a given record
df_similarity.iloc[0][df_similarity.iloc[0] > 0.5]

0       1.000000
3333    0.503106
3616    0.504519
4739    0.648122
4782    0.502276
7640    0.504422
7676    0.555534
Name: 0, dtype: float64

In [68]:
print(df.iloc[7676])

nct_id                                                                        NCT01885039
source                                  National Institutes of Health Clinical Center ...
brief_title                             Breast Tissue Study of Breast Diseases Among W...
overall_status                                                    Enrolling by invitation
study_type                                                                  Observational
study_first_posted                                                          June 24, 2013
last_update_posted                                                         March 21, 2019
phase                                                                                None
condition                                                                   Breast Cancer
condition_browse/mesh_term                                               Breast Neoplasms
intervention_browse/mesh_term                                                        None
detailed_d

In [69]:
print(df.iloc[0])

nct_id                                                                        NCT00058058
source                                      American College of Radiology Imaging Network
brief_title                             Magnetic Resonance Imaging in Women Recently D...
overall_status                                                                  Completed
study_type                                                                 Interventional
study_first_posted                                                          April 9, 2003
last_update_posted                                                          July 27, 2018
phase                                                                                 N/A
condition                                                                   Breast Cancer
condition_browse/mesh_term                                               Breast Neoplasms
intervention_browse/mesh_term                                                        None
detailed_d